# GoogLeNet - Inception v1

Paper: [Going Deeper with Convolutions - Christian Szegedy et al.](https://arxiv.org/pdf/1409.4842.pdf)

![arcitecture](https://files.slack.com/files-pri/T5BUP7CCT-FJN4XASRF/image.png)

In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, AveragePooling2D, Flatten, Dense, Dropout, Layer, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


## Inception Moldule

![arcitecture](https://i.stack.imgur.com/zTinD.png)

In [2]:
class Inception():
    ''' Inception Module

        Inputs filter dimensions of the different sub-layers and builds
        an inception module.
        
        Dimensions ordered the same as in Table 1. in the paper:
        
        Index 0 - 1×1
        Index 1 - 3×3 reduce
        Index 2 - 3×3 
        Index 3 - 5×5 reduce
        Index 4 - 5×5
        Index 5 - pool proj 
    '''
    
    def __init__(self, filter_dims):
        
        if len(filter_dims) != 6:
            raise ValueError('`filter_dims` must have a length of 6.')
        
        self.filter_dims = filter_dims
    
    def __call__(self, x):
        
        x1 = Conv2D(self.filter_dims[0], (1, 1), padding='same', activation='relu')(x)

        x2 = Conv2D(self.filter_dims[1], (1, 1), padding='same', activation='relu')(x)
        x2 = Conv2D(self.filter_dims[2], (3, 3), padding='same', activation='relu')(x2)

        x3 = Conv2D(self.filter_dims[3], (1, 1), padding='same', activation='relu')(x)
        x3 = Conv2D(self.filter_dims[4], (5, 5), padding='same', activation='relu')(x3)

        x4 = MaxPooling2D(pool_size=(3, 3), strides=(1,1), padding='same')(x)
        x4 = Conv2D(self.filter_dims[5], (1, 1), padding='same', activation='relu')(x4)

        x = Concatenate(axis=3)([x1, x2, x3, x4])

        return x

In [3]:
K.clear_session()

inputs = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(inputs)
x = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
x = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(x)
x = Inception([64, 96, 128, 16, 32, 32])(x)
x = Inception([128, 128, 192, 32, 96, 64])(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception([192, 96, 208, 16, 48, 64])(x)

x1 = AveragePooling2D((5, 5), strides=(3, 3))(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(1000, activation='softmax')(x1)

x = Inception([160, 112, 224, 24, 64, 64])(x)
x = Inception([128, 128, 256, 24, 64, 64])(x)
x = Inception([112, 144, 288, 32, 64, 64])(x)

x2 = AveragePooling2D((5, 5), strides=(3, 3))(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(1000, activation='softmax')(x2)

x = Inception([256, 160, 320, 32, 128, 128])(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception([256, 160, 320, 32, 128, 128])(x)
x = Inception([384, 192, 384, 48, 128, 128])(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(1000, activation='softmax')(x)

googlenet = Model(inputs=inputs, outputs=[x, x1, x2])
googlenet.compile(optimizer='sgd', loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
googlenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (